# Building Micrograd

Tiny autograd engine - implements backpropogation (reverse-mode-autodiff) over a dynamically built DAG (engine.py)

and a small neural network library (nn.py) on top of micrograd engine with a Pytorch-like API

Both are tiny

The DAG only operates over scalar values (rather than vectors) - so we chop up each nueron into all of its individual tiny adds adn multiplies
- Simplifies things to help you grok backpropogation 

If you want to train bigger networks you need to use tensors - this helps computation but none of the math actually changes
 - Tensors are just "arrays" of scalars - large arrays allow us to take advangage of parrallism of computer - but math is the same

So start with scalars to grok backpropogation

Everything else is just efficiency (that being said there is a lot to efficiency hahaahhaha)

Backpropogation - recursively applies the chain rule so can find the derivative of g with respect to all of the different nodes

derivate tells us how the nodes affect the output - obviously this is useful to train the network

Backpropogation is much more general than neural networks - it jsut happens that it is very useful to train neural nets 

In [1]:
import math
import numpy as np
import matplotlib.pyplot as plt
import random
%matplotlib inline

import torch

# see micrograd 1 & 2 for walkthrough of all the steps


In [2]:
class Value:

    def __init__(self,data, _children = (), _op = (), label = (), parent_grad = ()):
        self.data = data
        self._prev = set(_children)
            # _children will be a tuple but will be held as a set in the class _prev - just done for efficiency
        self._op = _op
        self.label = label

        #derivative
        self.grad = 0.0 # initialized at 0 - assume that value does not effect output at the start
        
        #backpop funciton
        self._backward = lambda: None

    def __repr__(self):
        #python interally uses this repr function to return this string when the object is called on its own
        return f"Value(data={self.data})"
    
    #need to define addition
    def __add__(self,other):
        #check if other is a value object and make it one if not
        other = other if isinstance(other, Value) else Value(other)

        #use special underscore emthods to define operators in pytho
        # e.g. if we od a + b, python will interally do a.__add__(b)
        out = Value(self.data + other.data, _children = (self,other), _op = "+" )#this operator works as it is operating on self.data which is jsut a python number rather than our value class, second part is feeding in the _children expression to the new value obejct
        
        # definint g the function that propgates the gradient
        def _backward():
            # we want to propogate outs.grad and change self.grad and other.grad
            self.grad += 1.0 * out.grad
            other.grad += 1.0 * self.grad

            # #recursive call ?not implemented
            # self._backward()
            # other._backward()
        out._backward = _backward

        return out
    
    #same for multiplication
    def __mul__(self, other):
        #check if other is a value object and make it one if not
        other = other if isinstance(other, Value) else Value(other)

        out = Value(self.data * other.data, (self,other), "*") #this operator works as it is operating on self.data which is jsut a python number rather than our value class
        
        def _backward():
            self.grad += other.data * out.grad
            other.grad += self.data * out.grad

            # #recursive calls - think abotu variable scope it makes ense
            # self._backward()
            # other._backward()
        out._backward = _backward
        
        return out

    def __neg__(self): #-self
        return self * -1

    def __sub__(self,other): #self - other
        return self + (-other) #saves us rewriting addition #wowowowo
    
    def __rsub__(self,other): #other - self reverse multiplication
        #this is a fallback - if __sub__ doesnt work - so python will then check if the second term has a rmul
        # this seems super ineficient but i guess that is the point
        return self - other

    def __rmul__(self,other): #other * self reverse multiplication
        #this is a fallback - if __mul__ doesnt work - so python will then check if the second term has a rmul
        # this seems super ineficient but i guess that is the point
        return self * other

    def __radd__(self,other): #other + self reverse addition
        #this is a fallback - if __add__ doesnt work - so python will then check if the second term has a rmul
        # this seems super ineficient but i guess that is the point (python is clever and nice but slow)
        return self + other  
    
    def __truediv__(self,other):
        #division is a special case of exponentiation x**k a/b = a * b**-1
        return self * other**-1


    def exp(self):
        # app exponentiation
        x = self.data
        out = Value(math.exp(x), (self,), "exp") #only 1 child

        def _backward():
            #same pointer trickery as before
            self.grad += out.grad * out.data

        out._backward = _backward

        return out

    def __pow__(self,k):
        #other must be a int/float for now
        #otherwise math wont work for  aspecific case ??? cant backprop to the k value - makes sense
        assert isinstance(k, (int,float))

        out = Value(self.data**k, (self,), "**{}".format(k))

        def _backward():
            #calculus baby x**k = k*x**(k-1)
            # remember out.grad is the reverse chain rule component
            self.grad += out.grad * k*((self.data)**(k-1))

        out._backward = _backward

        return out



    #defined activation function
    def tanh(self):
        x = self.data
        t = (math.exp(2*x) -1) / (math.exp(2*x) + 1)
        out = Value(t, (self, ), "tanh") #only 1 child

        def _backward():
            
            self.grad += out.grad * (1 - t**2)
            
            #having a bit of trouble figuring out the variable scope here
            #this function is not kept as a string but insted references the object pointers used when initializing it
            # when we "set" the function below
            # the function references self from the current variable scope - inside the tanh funciton where self refers not to the out Value but to the value which will create out

            # some simple debugging
            # print whatever self is when reference - when we call out._backwards we will see it prints the current self rather than out itself
            # This sounds schizophrenic but makes sense if you think about it enough
            # when you 

            # I have read the docs and returned wiser
            # when you define the function you bind the function tot he local namespace and the function object 
            # containes reference tot eh current global namespace as the flobal namespace tot be used when the 
            # function is called
            # so when we define the function ehre it remembers this tanh() function as the namespace and within
            # this space self refers to the object that is to create out
            # MAEKS SESNE
            # https://docs.python.org/3/reference/compound_stmts.html#function
            # print(self)
            
            # recusrive call not implemented
            # self._backward()


        out._backward = _backward
        
        return out
    

    #also want to make the connective tissue of the expression - keep expression graphs simple pointers
    #_children argument added

    #also need to know what operation created each value
    #_op


    #also want a way to visualize these expressions
    #see cell below

    # add labels _label

    #add another variable which keeps track of the derivative of L with respect to that value

    # #my attempt at writing it - to compare with the GOAT
    # def backprop(self, _parent_grad = (), _op = (), _sibling = ()):
        
    #     # Dont use unnecesary parent grad variable
    #     # can convey all the information reqiured for recursive function in the function call
    #     # previuosly used self.parent_grad function - uneccesary

    #     print(self.label, _parent_grad, _op, _sibling)
        
    #     if _parent_grad == ():
    #         #base case
    #         self.grad = 1 
    #         #derivative with respect to itself will be 1 always

    #     else:
    #         # apply chain rule using the parent grad
    #         if _op == "*":
    #             self.grad = _parent_grad * _sibling.data # parent = c1 * c2 ; dparent/dc1 = c2
    #         elif _op == "+":
    #             self.grad = _parent_grad * 1 # parent = c1 + c2 ; dparent/dc1 = 1 + 0 = 1

    #     if self._prev != set():
    #         #each nodes is assumed to have 2 children
    #         child1 , child2 = self._prev
            
    #         #unnecessarily pass sibling node even if addition
    #         child1.backprop(_parent_grad = self.grad, _op = self._op, _sibling = child2)
    #         child2.backprop(_parent_grad = self.grad, _op = self._op, _sibling = child1)

    # Kaprpathy backpopogates through each operation
    # ?just changes the childern directly and hten backprops through - makes way more sense as can handle larger sums :)
    # ?still need to check base case 
    # Karpathy getting fucking clever and using almbda functions
    # written into the code of building a network he specifies how to backprop over it 
    # seems weird but we will trust the GOAT
    
    def backward(self):
        #actaul backprop function - builds topolical tree and applies _backward attribute on it
    
        # building a topological graph
        topo = []
        visited = set()
        def build_topo(v):
            if v not in visited:
                visited.add(v)
                for child in v._prev:
                    build_topo(child)       
                topo.append(v) #only adds itself after all the children are processed
        build_topo(self)

        self.grad = 1
        for node in reversed(topo):
            node._backward()
        
            

 
    




In [3]:
# writing up a little function that installs 

from graphviz import Digraph

def trace(root):
    # root = starting Value object to build tree from

    #builds a set of all nodes and edges in a graph
    nodes, edges = set(), set()
    def build(v):
        if v not in nodes:
            nodes.add(v)
            for child in v._prev:
                edges.add((child,v))
                build(child) #recursive call to repeat addition on each child
    build(root)
    return nodes, edges

def draw_dot(root):
    dot = Digraph(format = "svg", graph_attr={"rankdir":"LR"}) #LR = left to right

    nodes, edges = trace(root)

    for n in nodes:
        uid = str(id(n))
        # for any vlaue in hte graph, create a rectuangular ("record") node for it
        dot.node(name = uid, label = "{%s | data %.4f | grad %s}" % (n.label, n.data,n.grad), shape = "record")
        if n._op: #if n is the result of an opertaion
            #if this value is a result of some operation, creat an op node for it
            dot.node(name = uid + n._op, label = n._op)
            #and connect this node to it
            dot.edge(uid+n._op,uid)
    
    for n1, n2 in edges:
            # connect n1 to the op node of n2
            dot.edge(str(id(n1)), str(id(n2)) + n2._op)

    return dot

#draw_dot(L)


# **building out a neural net following the pytorch API**


In [4]:
class Neuron:
    def __init__(self, nin):
        #nin = number of inputs
        #randomly initializes weights and bias
        self.w = [Value(random.uniform(-1,1)) for _ in range(nin)] 
        self.b = Value(random.uniform(-1,1))

    def __call__(self,x):
        #funciton() = funciton.__call__()

        #return output = activation(w dot x + b)
        #dot product is essentially pairwise multiplication
        act = sum((wi*xi for wi,xi in zip(self.w,x)),self. b)  #zip combines 2 iterators and iterates over the tuples of hte 2 interators combined 
        #interesting that sum works on Value() objects, second argument = intiial value - for a bit of efficiency
        out = act.tanh()

        return out
    
    def parameters(self):
        #written this way to match pytorch
        #pytorch has a .parameters on every since NN module which does this
        #one subtle difference - in pytorch this returns the parameters tensors
        # for us we return theparamter scalars (actually value objects)
        # ?torch.Tensor is the equivalent to our Value() class hmmmm?
        return self.w + [self.b]

x = [2.0,3.0]
n = Neuron(2)
n(x)


Value(data=0.9522192962364153)

# Defining a MULTILAYER PERCEPTRON

literally the simplest neural network

In [5]:
class Layer:
    def __init__(self,nin,nout):
        #initialize "nout" neurons each with "nin" inputs
        self.neurons = [Neuron(nin) for _ in range(nout)]
    
    def __call__(self,x):
        #return forward pass of all neurons in the layer
        #"independently evaluate them"
        outs = [n(x) for n in self.neurons]
        return outs[0] if len(outs) == 1 else outs #THIS IS THE CONDITIONAL RETURN SYNTAX DON'T NEED 2 RETURNS
    
    def parameters(self):
        #layer is also a module so to match pytorch it will lhave parameters - every module has parameters

        # params = []
        # for neuron in self.neurons:
        #     ps = neuron.parameters()
        #     params.extend(ps)
        # return params
        #SHORTEN THIS TO......

        return [p for neuron in self.neurons for p in neuron.parameters()] #nested single list comprehension
 
class MLP:
    def __init__(self,nin,nouts):
        #nin = number of inputs - scalar
        #nout = list of number of neurons in each layer
        sz = [nin] + nouts #concatenante these 2 together
        self.layers = [Layer(sz[i],sz[i+1]) for i in range(len(nouts))] #itrate through creating multiple layers

    def __call__(self,x):
        #need to pass each output as an input into the next layer
        #simple as that
        for layer in self.layers:
            x = layer(x)
        return x
    
    def parameters(self):
        #same format as above
        return [p for layer in self.layers for p in layer.parameters()]



In [6]:
#initializing neural network
# x = [2.0,3.0,-1]
nn = MLP(3,[4,4,1])
# nn(x)   
# nn

#draw_dot(nn(x))

#create mock dataset
# index does Rows the Columns - think recursively - first index will extract the list of a certain row
# second index will index that list along its "columns"
# made to match the way we draw matrixes by hand i think
#Xij = ith row, jth column

xs = [
    [2.0,3.0,-1.0],
    [3.0,-1.0,0.5],
    [0.5,1.0,1.0],
    [1.0,1.0,-1.0]
]
#for 1D lists think of them as being rows! and a singel column so indexing will match
ys = [1.0,-1.0,-1.0,1.0] #desired targets




# the loss

in order to train a neural network we need to specify a loss function 

and goal is to minimize loss


## Mean squared error loss

Makes sense that you square output so that absolute distance is all that matters - ?im guessing absolute value isnt as good because it is not continuously differentiable


In [7]:
#initial predictions
ypred = [nn(x) for x in xs] 
ypred

# to calculate loss oyu need hte ground truth and your predictions
loss = sum([(yout - ygt)**2 for ygt, yout in zip(ys,ypred)])
loss

Value(data=4.415022721308082)

## minimizing loss

black fucking magic baby - loss is the final value object - so can just call loss.backward()

In [8]:
loss.backward()

In [9]:

#find the gradients
print(nn.layers[0].neurons[0].w[0].grad)
print(nn.layers[0].neurons[0].w[0].data)
#draw_dot(loss)

#convenience code to allow us to access hte parameters of hte neural network
#paramerts method for all modules
#nn.parameters()

0.5952601838498193
0.1361519103465756


# Optimizing

In [10]:
#beautiful
for p in nn.parameters():
    step_size = 0.01
    p.data += -1*step_size*p.grad #-1 as gradient points in direction of INCREASING the loss so need to flip it to MINIMIZE
    p.grad = 0

# gradients explode unless you zero them after each pass

# need to be careful about step size

We only know hte very local dependence of hte loss function with respoect tot he parameters - not the full structure of the loss function

if we step too far we might step into a part of the loss that is very different

# making the training loop more respectful

In [11]:
#initialize network and inputs and ygt
nn = MLP(3,[4,4,1])

#draw_dot(nn(x))

xs = [
    [2.0,3.0,-1.0],
    [3.0,-1.0,0.5],
    [0.5,1.0,1.0],
    [1.0,1.0,-1.0]
]

ys = [1.0,-1.0,-1.0,1.0] #desired targets

In [12]:
epochs = 20
for k in range(epochs):
    #forward pass
    ypred = [nn(x) for x in xs]
    loss = sum(((yout - ygt)**2 for yout, ygt in zip(ypred,ys)))

    # I WAS RIGHT - NEED TO ZERO_GRAD() before backward pass
    for p in nn.parameters():
        p.grad = 0.0

    #backward pass
    loss.backward()

    # update
    for p in nn.parameters():
        step_size = 0.1
        p.data += -step_size * p.grad

    #print results
    print(k,loss.data)

0 7.070990741078284
1 3.678151078355111
2 1.6489472922785593
3 1.818566160511913
4 5.7457073389611715
5 3.7286925208922357
6 0.005640556411941524
7 0.005487893341849249
8 0.0053558749815995205
9 0.005206939809954647
10 0.005069744322705821
11 0.004955103578424387
12 0.00483189858059274
13 0.00472100679037112
14 0.004618359936641804
15 0.0045171994397072155
16 0.004420803289129852
17 0.004330840146888669
18 0.004243356804373374
19 0.004165299500813128
